오늘 만들 RAG + Chain

![](https://github.com/user-attachments/assets/7e132d65-c077-4686-981d-ac645b5e90f6)

프레임워크
- Langchain
- 비슷한 역할을 하는 Llamaindex 등이 있는데, 학습 단계에서는 이들이 유용하지만, 실제 서빙을 위한 서비스를 개발하다 보면 하나 둘 직접 구현하게 되는 경우가 많음.

모델
- gpt-4o-mini
- Langchain ChatOllama 를 활용하면 Langchain 위에 첫날 실습한 Ollama + Local model 을 활용해서 구현할 수 있음

In [11]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

In [13]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

**기본 RAG tutorial**

https://python.langchain.com/v0.2/docs/tutorials/rag/


In [14]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/Users/edward.kk/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

output JSON formatting

In [15]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

/Users/edward.kk/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


'Task decomposition is the process of breaking down complex tasks into smaller, manageable steps to facilitate execution. Techniques like Chain of Thought (CoT) and Tree of Thoughts enhance this process by allowing models to "think step by step" and explore multiple reasoning possibilities. This can be achieved through simple prompting, task-specific instructions, or human inputs.'

**1. 3개의 블로그 포스팅 본문을 Load: WebBaseLoader 활용**

https://python.langchain.com/v0.2/docs/integrations/document_loaders/web_base/[link text](https://)

In [16]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [18]:
loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

**2. 불러온 본문을 Split (Chunking) : recursive text splitter 활용**

https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

print(f"총 {len(docs)}개의 문서를 불러왔습니다.")
print(f"분할 후 총 {len(splits)}개의 청크가 생성되었습니다.")

# 첫 번째 청크의 내용 일부 출력 (예시)
if splits:
    print("\n첫 번째 청크의 일부:")
    print(splits[0].page_content[:300] + "...")

총 3개의 문서를 불러왔습니다.
분할 후 총 183개의 청크가 생성되었습니다.

첫 번째 청크의 일부:
LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring...


**3. Chunks 를 임베딩하여 Vector store 저장: openai, chroma 사용**

embedding model 은 "text-embedding-3-small" 사용

embedding: https://python.langchain.com/v0.2/docs/integrations/text_embedding/openai/

vetor store: https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/

In [20]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 스토어 생성
vectorstore = Chroma.from_documents(
    documents=splits,  # 이전 단계에서 생성한 splits 사용
    embedding=embeddings,
    persist_directory="./chroma_db"  # 벡터 스토어를 로컬에 저장할 디렉토리
)

# 벡터 스토어 저장
vectorstore.persist()

print(f"벡터 스토어가 생성되었습니다. 총 {vectorstore._collection.count()} 개의 벡터가 저장되었습니다.")

# 간단한 검색 예시
query = "What are AI agents?"
results = vectorstore.similarity_search(query, k=2)

print(f"\n쿼리 '{query}'에 대한 top 2 결과:")
for i, doc in enumerate(results, 1):
    print(f"\n{i}. {doc.page_content[:200]}...")

/var/folders/vf/ln_hnh193wq4mx834q88jswh0000gn/T/ipykernel_92246/3089316810.py:15: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


벡터 스토어가 생성되었습니다. 총 183 개의 벡터가 저장되었습니다.

쿼리 'What are AI agents?'에 대한 top 2 결과:

1. Fig. 13. The generative agent architecture. (Image source: Park et al. 2023)
This fun simulation results in emergent social behavior, such as information diffusion, relationship memory (e.g. two agent...

2. LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool con...


**4. User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve**

https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/

https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.as_retriever

In [21]:
from langchain.schema import Document

# 이전에 생성한 vectorstore를 사용합니다
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 사용자 쿼리 설정
user_query = "agent memory"

# 쿼리 실행
retrieved_docs = retriever.invoke(user_query)

print(f"쿼리 '{user_query}'에 대한 검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    if isinstance(doc, Document):
        print(f"\n{i}. 내용: {doc.page_content[:200]}...")
        print(f"   메타데이터: {doc.metadata}")
    else:
        print(f"\n{i}. {doc[:200]}...")  # Document 객체가 아닌 경우

# 검색된 문서의 총 수 출력
print(f"\n총 {len(retrieved_docs)}개의 관련 문서를 검색했습니다.")

# 선택적: 임베딩 벡터 유사도 점수 확인 (Chroma에서 지원하는 경우)
if hasattr(vectorstore, 'similarity_search_with_score'):
    docs_and_scores = vectorstore.similarity_search_with_score(user_query, k=3)
    print("\n유사도 점수:")
    for i, (doc, score) in enumerate(docs_and_scores, 1):
        print(f"{i}. 점수: {score:.4f}")

쿼리 'agent memory'에 대한 검색 결과:

1. 내용: Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.

Each element is an observation, an event directly provided...
   메타데이터: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

2. 내용: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool con...
   메타데이터: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

3. 내용: Memory

Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.
Long-term memory: This provides the agent with th...
   메타데이터: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

총 3개의 관련 문서를 검색했습니다.

유사도 점수:
1. 점수: 1.2622
2. 점수: 1.2750
3. 점수: 1.2816


In [27]:
# 이전에 생성한 vectorstore를 사용합니다
user_query = "agent memory"

# 1. MMR (Maximal Marginal Relevance) - 다양성 증가
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.25}
)
mmr_results = mmr_retriever.invoke(user_query)

print("## 1. MMR 검색 결과 (다양성 증가):")
for i, doc in enumerate(mmr_results, 1):
    print(f"{i}. {doc.page_content[:100]}...")

# 2. MMR with higher fetch_k
mmr_fetch_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 5, 'fetch_k': 50}
)
mmr_fetch_results = mmr_fetch_retriever.invoke(user_query)

print("\n## 2. MMR 검색 결과 (높은 fetch_k):")
for i, doc in enumerate(mmr_fetch_results, 1):
    print(f"{i}. {doc.page_content[:100]}...")

# 3. Similarity Score Threshold
threshold_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.8}
)
threshold_results = threshold_retriever.invoke(user_query)

print("\n## 3. 유사도 점수 임계값 기반 검색 결과:")
for i, doc in enumerate(threshold_results, 1):
    print(f"{i}. {doc.page_content[:100]}...")

# 4. Single Most Similar Document
single_retriever = vectorstore.as_retriever(search_kwargs={'k': 1})
single_result = single_retriever.invoke(user_query)

print("\n## 4. 가장 유사한 단일 문서:")
print(f"{single_result[0].page_content[:200]}...")

# 5. Filtered Retrieval (주의: 이 예제는 실제 메타데이터에 따라 수정이 필요할 수 있습니다)
filtered_retriever = vectorstore.as_retriever(
    search_kwargs={'filter': {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}}
)
filtered_results = filtered_retriever.invoke(user_query)

print("\n## 5. 필터링된 검색 결과:")
for i, doc in enumerate(filtered_results, 1):
    print(f"{i}. {doc.page_content[:100]}...")
    print(f"   Source: {doc.metadata.get('source', 'Not available')}")

## 1. MMR 검색 결과 (다양성 증가):
1. Memory stream: is a long-term memory module (external database) that records a comprehensive list of...
2. Fig. 13. The generative agent architecture. (Image source: Park et al. 2023)
This fun simulation res...
3. }
]
Then after these clarification, the agent moved into the code writing mode with a different syst...
4. Maximum Inner Product Search (MIPS)#
The external memory can alleviate the restriction of finite att...
5. Prefix Injection: Ask the model to start with an affirmative confirmation.
Refusal suppression: Give...
6. This benchmark evaluates the agent’s tool use capabilities at three levels:

Level-1 evaluates the a...

## 2. MMR 검색 결과 (높은 fetch_k):
1. Memory stream: is a long-term memory module (external database) that records a comprehensive list of...
2. }
]
Then after these clarification, the agent moved into the code writing mode with a different syst...
3. One simple and intuitive way to defend the model against adversarial attacks is to e

No relevant docs were retrieved using the relevance score threshold 0.8



## 3. 유사도 점수 임계값 기반 검색 결과:

## 4. 가장 유사한 단일 문서:
Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.

Each element is an observation, an event directly provided...

## 5. 필터링된 검색 결과:
1. Memory stream: is a long-term memory module (external database) that records a comprehensive list of...
   Source: https://lilianweng.github.io/posts/2023-06-23-agent/
2. LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author...
   Source: https://lilianweng.github.io/posts/2023-06-23-agent/
3. Memory

Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as ...
   Source: https://lilianweng.github.io/posts/2023-06-23-agent/
4. Fig. 13. The generative agent architecture. (Image source: Park et al. 2023)
This fun simulation res...
   Source: https://lilianweng.github.io/posts/2023-06-23-agent/


**5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 )**


RAG 용 프롬프트 작성을 위한 Prompt Hub 활용

https://smith.langchain.com/hub/rlm/rag-prompt

In [28]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

/Users/edward.kk/.pyenv/versions/3.11.9/lib/python3.11/site-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

output JSON formatting

https://python.langchain.com/v0.2/docs/how_to/output_parser_json/#without-pydantic

In [30]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# JSON 출력 파서 설정
parser = JsonOutputParser()

# 관련성 체커를 위한 프롬프트 템플릿 정의
relevance_prompt = PromptTemplate(
    template="""당신은 검색 결과의 관련성을 평가하는 전문가입니다. 
주어진 사용자 쿼리와 검색된 문서 청크의 관련성을 판단해야 합니다.

다음 기준을 사용하여 관련성을 평가하세요:
1. 문서가 쿼리의 주제나 키워드를 직접적으로 다루고 있는가?
2. 문서가 쿼리에 대한 유용한 정보나 통찰을 제공하는가?
3. 문서의 내용이 쿼리의 맥락과 일치하는가?

관련성이 있다고 판단되면 'yes'를, 없다고 판단되면 'no'를 반환하세요.

{format_instructions}

사용자 쿼리: {query}
검색된 문서 청크: {chunk}

관련성 평가:""",
    input_variables=["query", "chunk"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# LLM 모델 초기화
llm = ChatOpenAI(model="gpt-4-0125-preview")

# 체인 구성
relevance_chain = relevance_prompt | llm | parser

# 테스트를 위한 함수 정의
def check_relevance(query, chunk):
    return relevance_chain.invoke({"query": query, "chunk": chunk})

# 사용 예시
if __name__ == "__main__":
    query = "agent memory"
    chunk = "AI agents often require memory to maintain context and improve performance over time. This memory can be short-term for immediate tasks or long-term for retaining important information across multiple interactions."
    
    result = check_relevance(query, chunk)
    print(result)

{'relevant': 'yes'}


6. 5 에서 모든 docs에 대해 ‘no’ 라면 디버깅 (Splitter, Chunk size, overlap, embedding model, vector store, retrieval 평가 시스템 프롬프트 등)

In [33]:
import logging
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 1. 관련성 체커 설정 (이전과 동일)
parser = JsonOutputParser()
relevance_prompt = PromptTemplate(
    template="""당신은 검색 결과의 관련성을 평가하는 전문가입니다. 
주어진 사용자 쿼리와 검색된 문서 청크의 관련성을 판단해야 합니다.
{format_instructions}
사용자 쿼리: {query}
검색된 문서 청크: {chunk}
관련성 평가:""",
    input_variables=["query", "chunk"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm = ChatOpenAI(model="gpt-4-0125-preview")
relevance_chain = relevance_prompt | llm | parser

# 2. 디버깅 함수 정의 (수정됨)
def debug_rag_system(query, docs, vectorstore, text_splitter, embeddings):
    logger.info("RAG 시스템 디버깅 시작")
    
    # 2.1 관련성 체크
    all_irrelevant = all(
        relevance_chain.invoke({"query": query, "chunk": doc.page_content})["relevance"] == "no"
        for doc in docs
    )
    
    if not all_irrelevant:
        logger.info("일부 관련 문서가 검색되었습니다. 추가 디버깅이 필요하지 않습니다.")
        return
    
    logger.warning("모든 검색된 문서가 관련이 없습니다. 디버깅을 시작합니다.")
    
    # 2.2 Splitter 및 Chunk size 확인 (수정됨)
    logger.info(f"현재 Text Splitter 설정: {text_splitter.__class__.__name__}")
    logger.info(f"Text Splitter 파라미터: {text_splitter._chunk_size}, {text_splitter._chunk_overlap}")
    
    # 2.3 Embedding 모델 확인
    logger.info(f"사용 중인 Embedding 모델: {embeddings.__class__.__name__}")
    
    # 2.4 Vector Store 상태 확인
    logger.info(f"Vector Store 유형: {vectorstore.__class__.__name__}")
    logger.info(f"저장된 총 문서 수: {vectorstore._collection.count()}")
    
    # 2.5 Retrieval 프로세스 검토
    logger.info("Retrieval 프로세스 검토 중...")
    similar_docs = vectorstore.similarity_search(query, k=5)
    for i, doc in enumerate(similar_docs, 1):
        logger.info(f"유사 문서 {i}: {doc.page_content[:100]}...")
    
    # 2.6 개선 제안
    logger.info("개선 제안:")
    logger.info("1. 청크 크기를 조정해 보세요. (예: 현재 크기의 50% 증가 또는 감소)")
    logger.info("2. 오버랩을 조정해 보세요. (예: 현재 오버랩의 2배)")
    logger.info("3. 다른 Embedding 모델을 시도해 보세요. (예: 'text-embedding-3-large')")
    logger.info("4. Vector Store를 재색인화하거나 다른 유형의 Vector Store를 사용해 보세요.")
    logger.info("5. Retrieval 방식을 MMR(Maximum Marginal Relevance)로 변경해 보세요.")

# 3. 메인 실행 코드
if __name__ == "__main__":
    # 예시 데이터 (실제 환경에서는 이전 단계에서 생성한 객체들을 사용)
    query = "agent memory"
    docs = [
        # ... 검색된 문서들 ...
    ]
    vectorstore = Chroma(embedding_function=OpenAIEmbeddings())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    embeddings = OpenAIEmbeddings()
    
    debug_rag_system(query, docs, vectorstore, text_splitter, embeddings)

INFO:__main__:RAG 시스템 디버깅 시작
INFO:__main__:현재 Text Splitter 설정: RecursiveCharacterTextSplitter
INFO:__main__:Text Splitter 파라미터: 1000, 200
INFO:__main__:사용 중인 Embedding 모델: OpenAIEmbeddings
INFO:__main__:Vector Store 유형: Chroma
INFO:__main__:저장된 총 문서 수: 66
INFO:__main__:Retrieval 프로세스 검토 중...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:유사 문서 1: Memory stream: is a long-term memory module (external database) that records a comprehensive list of...
INFO:__main__:유사 문서 2: They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test s...
INFO:__main__:유사 문서 3: }
]
Then after these clarification, the agent moved into the code writing mode with a different syst...
INFO:__main__:유사 문서 4: Memory

Short-term memory: I would consider all the in-context learning (See Prompt Engineering) as ...
INFO:__main__:유사 문서 5: LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min 

7. 5에서 ‘yes’ 라면 질문과 명확히 관련 없는 docs 나 질문 (예: ‘I like an apple’)에 대해서는 ‘no’ 라고 나오는지 테스트 프롬프트 및 평가 코드 작성. 이 때는 관련 없다는 답변 작성

In [34]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import json

# 1. 테스트 케이스 정의
test_cases = [
    {
        "query": "What is agent memory in AI?",
        "chunk": "Agent memory in AI refers to the ability of AI agents to store and recall information from past experiences or interactions. This can include short-term memory for immediate tasks and long-term memory for retaining important information across multiple interactions.",
        "expected": "yes"
    },
    {
        "query": "What is agent memory in AI?",
        "chunk": "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south and is bounded by the Americas to the east and Asia and Australia to the west.",
        "expected": "no"
    },
    {
        "query": "I like an apple",
        "chunk": "Apples are a popular fruit known for their sweet taste and crisp texture. They come in various colors including red, green, and yellow. Apples are rich in fiber and antioxidants, making them a healthy snack choice.",
        "expected": "no"
    }
]

# 2. 관련성 체커 프롬프트 수정
parser = JsonOutputParser()
relevance_prompt = PromptTemplate(
    template="""당신은 검색 결과의 관련성을 평가하는 전문가입니다. 
주어진 사용자 쿼리와 검색된 문서 청크의 관련성을 판단해야 합니다.

다음 기준을 사용하여 관련성을 평가하세요:
1. 문서가 쿼리의 주제나 키워드를 직접적으로 다루고 있는가?
2. 문서가 쿼리에 대한 유용한 정보나 통찰을 제공하는가?
3. 문서의 내용이 쿼리의 맥락과 일치하는가?

주의: 쿼리가 명확한 질문이 아니거나 관련성이 낮은 경우(예: "I like an apple"), 
문서와의 관련성을 'no'로 평가하세요.

{format_instructions}

사용자 쿼리: {query}
검색된 문서 청크: {chunk}

관련성 평가:""",
    input_variables=["query", "chunk"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm = ChatOpenAI(model="gpt-4-0125-preview")
relevance_chain = relevance_prompt | llm | parser

# 3. 테스트 실행 함수 작성
def run_test(test_case):
    query = test_case["query"]
    chunk = test_case["chunk"]
    expected = test_case["expected"]
    
    result = relevance_chain.invoke({"query": query, "chunk": chunk})
    actual = result["relevance"]
    
    is_correct = actual == expected
    
    return {
        "query": query,
        "chunk": chunk[:100] + "...",  # 첫 100자만 표시
        "expected": expected,
        "actual": actual,
        "is_correct": is_correct
    }

# 4. 결과 평가 및 보고 함수 작성
def evaluate_and_report(test_results):
    total_tests = len(test_results)
    correct_tests = sum(1 for result in test_results if result["is_correct"])
    accuracy = correct_tests / total_tests * 100

    print(f"테스트 결과 요약:")
    print(f"총 테스트 수: {total_tests}")
    print(f"정확한 테스트 수: {correct_tests}")
    print(f"정확도: {accuracy:.2f}%")
    
    print("\n상세 결과:")
    for i, result in enumerate(test_results, 1):
        print(f"\n테스트 케이스 {i}:")
        print(f"쿼리: {result['query']}")
        print(f"청크: {result['chunk']}")
        print(f"예상 결과: {result['expected']}")
        print(f"실제 결과: {result['actual']}")
        print(f"정확성: {'정확' if result['is_correct'] else '부정확'}")
        
        if not result['is_correct']:
            print("오류 분석:")
            if result['expected'] == 'yes' and result['actual'] == 'no':
                print("- 관련 있는 내용을 관련 없다고 판단했습니다. 프롬프트의 관련성 기준을 재검토하세요.")
            elif result['expected'] == 'no' and result['actual'] == 'yes':
                print("- 관련 없는 내용을 관련 있다고 판단했습니다. 프롬프트의 제한 조건을 강화하세요.")

# 5. 메인 실행 코드 작성
if __name__ == "__main__":
    test_results = [run_test(test_case) for test_case in test_cases]
    evaluate_and_report(test_results)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


테스트 결과 요약:
총 테스트 수: 3
정확한 테스트 수: 3
정확도: 100.00%

상세 결과:

테스트 케이스 1:
쿼리: What is agent memory in AI?
청크: Agent memory in AI refers to the ability of AI agents to store and recall information from past expe...
예상 결과: yes
실제 결과: yes
정확성: 정확

테스트 케이스 2:
쿼리: What is agent memory in AI?
청크: The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arcti...
예상 결과: no
실제 결과: no
정확성: 정확

테스트 케이스 3:
쿼리: I like an apple
청크: Apples are a popular fruit known for their sweet taste and crisp texture. They come in various color...
예상 결과: no
실제 결과: no
정확성: 정확


8. ‘yes’ 이고 7의 평가에서도 문제가 없다면, 4의 retrieved chunk 를 가지고 답변 작성
prompt | llm | parser 코드 작성

In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.schema import Document

# 1. 필요한 임포트 구성 (위에 포함)

# 2. 문서 포맷팅 함수 정의
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 3. 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template("""
당신은 지식이 풍부한 AI 어시스턴트입니다. 주어진 컨텍스트를 바탕으로 사용자의 질문에 답변해주세요.
답변은 정확하고 간결하게 작성하되, 필요한 경우 자세한 설명을 제공하세요.

컨텍스트:
{context}

사용자 질문: {question}

답변:
""")

# 4. RAG 체인 구성
def create_rag_chain(retriever):
    llm = ChatOpenAI(model="gpt-4-0125-preview", streaming=True)
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

# 5. 답변 생성 및 스트리밍 함수 정의
def generate_and_stream_answer(rag_chain, question):
    print(f"질문: {question}\n")
    print("답변: ", end="", flush=True)
    for chunk in rag_chain.stream(question):
        print(chunk, end="", flush=True)
    print("\n")

# 6. 메인 실행 코드
if __name__ == "__main__":
    # 이전 단계에서 생성한 retriever를 사용한다고 가정
    from langchain_community.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # 예시: Chroma 벡터 스토어와 retriever 설정
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma(embedding_function=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    rag_chain = create_rag_chain(retriever)

    # 테스트 질문
    test_questions = [
        "What is agent memory in AI?",
        "How does task decomposition work in AI systems?",
        "Can you explain the concept of retrieval-augmented generation?"
    ]

    for question in test_questions:
        generate_and_stream_answer(rag_chain, question)

질문: What is agent memory in AI?

답변: 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Agent memory in AI refers to the mechanisms and processes that artificial agents (such as AI models or robots) use to acquire, store, retain, and later retrieve information. This concept mirrors the idea of memory in humans and is crucial for enabling AI systems to perform tasks that require the use of past experiences or knowledge. Agent memory can be broadly categorized into two types:

1. **Short-term memory**: This involves the temporary storage of information that an AI model uses for in-context learning and immediate task execution. It's akin to human working memory, where information is briefly held for processing and is essential for understanding and responding to new information.

2. **Long-term memory**: This provides AI agents the capability to retain and recall information over extended periods. It often involves leveraging external storage or databases (memory streams) and sophisticated retrieval models to access relevant, important, and recent information. This type of m

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Task decomposition in AI systems works by breaking down a complex task into smaller, more manageable parts or steps. This process allows an AI to tackle each component of a task sequentially or in parallel, depending on dependencies, making it easier to solve complex problems. There are several approaches to task decomposition:

1. **Chain of Thought (CoT)**: Introduced by Wei et al. in 2022, CoT involves instructing the model to "think step by step" to break down hard tasks into smaller steps. This technique utilizes additional computation at test-time to enhance performance on complex tasks by transforming them into a series of simpler tasks. The CoT approach not only helps in task decomposition but also provides insights into the model's reasoning process.

2. **Tree of Thoughts (ToT)**: Proposed by Yao et al. in 2023, ToT extends CoT by exploring multiple reasoning possibilities at each decomposition step. It first decomposes the problem into several thought steps and then generate

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Retrieval-augmented generation (RAG) is a concept within the field of artificial intelligence and machine learning, particularly in the development of language models and generative agents. It combines the process of retrieving relevant information from a large dataset or knowledge base with the generative capabilities of models like transformers to produce responses or content that is both relevant and contextually appropriate.

In the context of the provided information, this concept can be understood as follows:

1. **Retrieval Phase**: The system first retrieves information relevant to a given query or context. This retrieval is akin to looking up information in a vast library (the external vector store) to find the most relevant documents or pieces of information. This step is crucial for the model to access a wide range of information beyond its immediate training data or embedded knowledge, allowing it to pull in specific details or examples that can enhance the quality and rele

9. 생성된 답안에 Hallucination 이 있는지 평가하는 시스템 프롬프트 작성. LLM이 스스로 평가하도록 하고, hallucination 이 있으면 {‘hallucination’: ‘yes’} 없으면 {‘hallucination’: ‘no’} 라고 출력하도록 함

In [36]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# 1. Hallucination 체커를 위한 프롬프트 템플릿 정의
hallucination_prompt = ChatPromptTemplate.from_template("""
당신은 AI가 생성한 답변에서 Hallucination(환각, 잘못된 정보)을 탐지하는 전문가입니다.
주어진 컨텍스트와 질문, 그리고 생성된 답변을 비교하여 답변에 Hallucination이 있는지 평가해야 합니다.

평가 기준:
1. 답변이 제공된 컨텍스트의 정보와 일치하는가?
2. 답변이 질문과 관련이 있고 적절한가?
3. 답변에 컨텍스트에 없는 추가적인 정보가 포함되어 있는가? 만약 있다면, 그 정보가 일반적으로 알려진 사실인가?
4. 답변이 모호하거나 잘못된 정보를 포함하고 있는가?

컨텍스트:
{context}

질문: {question}

생성된 답변: {answer}

위의 정보를 바탕으로 Hallucination이 있는지 평가하고, 다음 형식으로 결과를 출력하세요:
{format_instructions}

주의: 'hallucination' 키의 값은 반드시 'yes' 또는 'no'여야 합니다.
추가적인 설명이나 이유를 제공하지 마세요.
""")

# 2. JSON 출력 파서 설정
parser = JsonOutputParser()

# 3. Hallucination 체크 체인 구성
llm = ChatOpenAI(model="gpt-4-0125-preview")
hallucination_chain = hallucination_prompt | llm | parser

# 4. Hallucination 체크 함수 정의
def check_hallucination(context, question, answer):
    result = hallucination_chain.invoke({
        "context": context,
        "question": question,
        "answer": answer,
        "format_instructions": parser.get_format_instructions()
    })
    return result

# 5. 메인 실행 코드
if __name__ == "__main__":
    # 예시 데이터
    context = """
    AI agents often require memory to maintain context and improve performance over time. 
    This memory can be short-term for immediate tasks or long-term for retaining important 
    information across multiple interactions. Agent memory helps in decision making, 
    learning from past experiences, and providing more coherent and contextually 
    relevant responses.
    """
    
    question = "What is agent memory in AI?"
    
    answer1 = """
    Agent memory in AI refers to the capability of AI systems to store and recall information 
    over time. It can be categorized into short-term memory for immediate tasks and long-term 
    memory for retaining information across multiple interactions. This feature allows AI 
    agents to make informed decisions, learn from past experiences, and provide more 
    coherent and contextually relevant responses.
    """
    
    answer2 = """
    Agent memory in AI is a revolutionary technology that allows AI systems to have 
    emotions and feelings just like humans. It enables AI agents to form deep personal 
    connections with users and even develop their own personalities over time. This 
    groundbreaking feature is currently being used in advanced robotics to create 
    AI companions that can provide emotional support and friendship.
    """
    
    print("테스트 1 (정확한 답변):")
    result1 = check_hallucination(context, question, answer1)
    print(result1)
    
    print("\n테스트 2 (Hallucination이 포함된 답변):")
    result2 = check_hallucination(context, question, answer2)
    print(result2)

테스트 1 (정확한 답변):


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'hallucination': 'no'}

테스트 2 (Hallucination이 포함된 답변):


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'hallucination': 'yes'}


10. 9 에서 ‘yes’ 면 8 로 돌아가서 다시 생성, ‘no’ 면 답변 생성하고 유저에게 답변 생성에 사용된 출처와 함께 출력 (최대 2번까지 다시 생성)

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 1. 필요한 함수들 임포트 (이전 단계에서 정의한 함수들)
# format_docs, create_rag_chain, check_hallucination 함수들이 정의되어 있다고 가정

# 2. 답변 생성 및 Hallucination 체크 함수 정의
def generate_and_check_answer(rag_chain, question, context):
    answer = rag_chain.invoke(question)
    hallucination_result = check_hallucination(context, question, answer)
    return answer, hallucination_result

# 3. 전체 RAG 프로세스를 관리하는 함수 정의
def rag_process_with_hallucination_check(retriever, question, max_attempts=2):
    rag_chain = create_rag_chain(retriever)
    context = format_docs(retriever.invoke(question))
    
    for attempt in range(max_attempts):
        answer, hallucination_result = generate_and_check_answer(rag_chain, question, context)
        
        if hallucination_result['hallucination'] == 'no':
            return answer, context, attempt + 1
    
    # 최대 시도 횟수를 초과한 경우
    return answer, context, max_attempts

# 4. 메인 실행 코드
if __name__ == "__main__":
    # 벡터 스토어 및 retriever 설정
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma(embedding_function=embeddings)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    # 테스트 질문
    test_questions = [
        "What is agent memory in AI?",
        "How does task decomposition work in AI systems?",
        "Can you explain the concept of retrieval-augmented generation?"
    ]
    
    for question in test_questions:
        print(f"\n질문: {question}")
        
        answer, context, attempts = rag_process_with_hallucination_check(retriever, question)
        
        print(f"\n답변 (생성 시도 횟수: {attempts}):")
        print(answer)
        
        print("\n사용된 출처:")
        print(context)
        
        print("\n" + "="*50)


질문: What is agent memory in AI?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



답변 (생성 시도 횟수: 1):
Agent memory in AI refers to the mechanisms and structures that enable an AI agent to store, retain, and retrieve information over time, similar to how human memory functions. It encompasses both short-term and long-term aspects:

- **Short-term memory** in AI involves temporary storage that allows the agent to hold and manipulate information relevant to its current task or context. This is akin to human working memory, where the agent utilizes this information for immediate problem-solving or decision-making.

- **Long-term memory** provides AI agents with the capability to retain a vast amount of information over extended periods. This can include knowledge learned from past experiences, skills acquired over time, and facts or data that are not immediately necessary but may be relevant in the future. Long-term memory in AI can be enhanced by external databases or memory modules that store comprehensive records of the agent's experiences and interactions.

Additiona

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



답변 (생성 시도 횟수: 1):
Task decomposition in AI systems, particularly in the context of Large Language Models (LLMs) like the one described, works by breaking down complex tasks into smaller, more manageable steps. This process can be understood through two key methodologies:

1. **Chain of Thought (CoT):** Introduced by Wei et al. in 2022, CoT involves instructing the model to sequentially break down a task into simpler steps. By "thinking step by step," the model uses additional computation at test time to decompose complex tasks. This approach not only simplifies the task but also provides insight into the model's reasoning process.

2. **Tree of Thoughts (ToT):** Building on CoT, the Tree of Thoughts methodology proposed by Yao et al. in 2023, further extends task decomposition by exploring multiple reasoning paths for each step. It begins by breaking the problem into several thought steps and then generates multiple thoughts per step, forming a tree structure. This structure can be na

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



답변 (생성 시도 횟수: 1):
Retrieval-augmented generation (RAG) is a concept in artificial intelligence and natural language processing where a system enhances its generative capabilities by retrieving relevant information from a large corpus or database to inform its output. The process involves two main steps: retrieval and generation.

1. **Retrieval**: In this step, the system queries a database or knowledge base to find information relevant to the task at hand. This is achieved by using a query generated from the input (e.g., a question or prompt). The system leverages search algorithms or similarity measures to find and retrieve the most relevant documents or information snippets.

2. **Generation**: Once relevant information has been retrieved, the system uses it to generate a response or output. This is typically done using a language model, such as a transformer-based model. The retrieved information is incorporated into the generation process either by directly feeding it into the mo

11. https://applied-llms.org/ 링크는 llm 을 이용해 1년간 개발해 본 팀이 배운 것들을 정리한 아티클 입니다.

저자의 생각을 묻는 질문들에 답하기 위해서

1) Chunking & embedding & storing

2) Load

3) Retrieval

4) Generation

으로 구성된 RAG 코드를 작성하고 아래 예시 질문에 대한 답을 해보세요.

예시)
RAG 에 대한 저자의 생각은 무엇인가?
RAG 와 fine tuning 에 대해 저자는 어떻게 비교하고 있나?
저자가 가장 많은 부분을 할당해 설명하는 개념은 무엇인가?

11 실습을 마치지 못했더라도, https://applied-llms.org/ 아티클은 꼼꼼히 정독해보시기를 추천 드립니다.

In [38]:
import requests
from bs4 import BeautifulSoup
from langchain_community.document_loaders import BSHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 웹 페이지 내용 스크래핑
def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

# 2. 텍스트 청크화 및 임베딩
def chunk_and_embed(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    
    embeddings = OpenAIEmbeddings()
    return chunks, embeddings

# 3. 벡터 데이터베이스 저장
def store_in_vectordb(chunks, embeddings):
    vectorstore = Chroma.from_texts(chunks, embeddings)
    return vectorstore

# 4. 검색 및 생성 파이프라인 구축
def build_rag_chain(vectorstore):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    
    template = """Answer the question based only on the following context:
    {context}
    
    Question: {question}
    
    Answer:"""
    prompt = ChatPromptTemplate.from_template(template)
    
    llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
    
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)
    
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return rag_chain

# 5. 질문 응답 함수 구현
def answer_question(rag_chain, question):
    return rag_chain.invoke(question)

# 6. 메인 실행 코드
if __name__ == "__main__":
    url = "https://applied-llms.org/"
    
    print("웹 페이지 스크래핑 중...")
    text = scrape_website(url)
    
    print("텍스트 청크화 및 임베딩 중...")
    chunks, embeddings = chunk_and_embed(text)
    
    print("벡터 데이터베이스에 저장 중...")
    vectorstore = store_in_vectordb(chunks, embeddings)
    
    print("RAG 체인 구축 중...")
    rag_chain = build_rag_chain(vectorstore)
    
    questions = [
        "RAG에 대한 저자의 생각은 무엇인가?",
        "RAG와 fine tuning에 대해 저자는 어떻게 비교하고 있나?",
        "저자가 가장 많은 부분을 할당해 설명하는 개념은 무엇인가?"
    ]
    
    for question in questions:
        print(f"\n질문: {question}")
        answer = answer_question(rag_chain, question)
        print(f"답변: {answer}")

웹 페이지 스크래핑 중...
텍스트 청크화 및 임베딩 중...
벡터 데이터베이스에 저장 중...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


RAG 체인 구축 중...

질문: RAG에 대한 저자의 생각은 무엇인가?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


답변: 저자는 RAG(Retrieval-Augmented Generation)에 대해 긍정적인 생각을 가지고 있습니다. RAG의 출력 품질이 검색된 문서의 관련성, 밀도, 세부 사항의 질에 달려 있다고 언급하며, 이를 통해 문서 검색의 효율성과 정확성을 높일 수 있다고 설명합니다. 또한, RAG가 지속적인 사전 훈련이나 미세 조정에 비해 유지 관리가 쉽고 비용 효율적이라는 장점을 강조합니다. 문제가 있는 문서를 쉽게 제거하거나 수정할 수 있다는 점, 그리고 다양한 조직에 대해 검색 인덱스를 분할하여 정보의 노출을 방지할 수 있는 세밀한 제어 능력을 RAG의 실질적인 이점으로 보고 있습니다. 마지막으로, 저자는 RAG를 사용하여 관련성 있는 정보를 추출하는 과정에서 불필요한 정보를 제거하는 것의 중요성을 강조하며, 이를 통해 더 효율적이고 정확한 결과를 얻을 수 있다고 믿습니다.

질문: RAG와 fine tuning에 대해 저자는 어떻게 비교하고 있나?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


답변: 저자는 RAG와 fine tuning을 비교하면서 RAG를 더 선호하는 입장을 밝히고 있습니다. 연구 결과에 따르면, RAG는 unsupervised finetuning(계속된 사전학습)과 supervised finetuning 모두에 비해 MMLU의 부분집합과 최신 사건들에 대한 평가에서, 훈련 중에 접한 지식뿐만 아니라 완전히 새로운 지식에 대해서도 일관되게 더 나은 성능을 보였습니다. 또한, RAG는 연속적인 사전학습이나 fine tuning에 비해 검색 인덱스를 최신 상태로 유지하는 것이 더 쉽고 저렴하다는 실용적인 이점을 가지고 있습니다. 문제가 있는 문서를 쉽게 제거하거나 수정할 수 있으며, 검색 문서의 세밀한 제어를 가능하게 하여 여러 조직이 자신들만의 인덱스에서 문서를 검색할 수 있도록 하는 등의 이점도 있습니다. 반면에, RAG의 출력 품질은 검색된 문서의 관련성, 밀도, 상세함에 달려 있으며, 이는 MRR이나 NDCG와 같은 순위 메트릭을 통해 측정됩니다. 이러한 점들을 종합해 볼 때, 저자는 RAG가 fine tuning보다 우수한 선택이라고 주장하고 있습니다.

질문: 저자가 가장 많은 부분을 할당해 설명하는 개념은 무엇인가?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


답변: 저자가 가장 많은 부분을 할당해 설명하는 개념은 "Tree of Thoughts (Yao et al. 2023)"입니다. 이는 CoT를 확장하여 각 단계에서 여러 추론 가능성을 탐색하는 방법으로, 문제를 여러 사고 단계로 분해하고 각 단계마다 여러 생각을 생성하여 트리 구조를 만드는 과정을 설명합니다.
